In [2]:
!pip install SQLAlchemy psycopg2-binary


Conectando con la BD

In [3]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine(
    "postgresql+psycopg2://grupo2:0ber8qGOMdXCjDrVq6pjGOJrCxj0XWHz@"
    "dpg-d4r4osm3jp1c739r0i20-a.virginia-postgres.render.com/transtats"
    "?sslmode=require"
)

# Ver tablas públicas
tables = pd.read_sql("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
""", engine)

tables


,table_name
0,pg_stat_statements_info
1,l_cancellation
2,pg_stat_statements
3,transtats_historic
4,l_city_market_id
5,l_airline_id
6,l_state_fips
7,l_airport
8,l_unique_carriers
9,l_airport_id


**Análisis EDA**


El presente Análisis Exploratorio de Datos (EDA) tiene como objetivo comprender el comportamiento operativo de los vuelos comerciales en Estados Unidos, a partir del dataset TranStats del Bureau of Transportation Statistics. El análisis se centra en métricas descriptivas relacionadas con tiempos de vuelo, demoras, distancia y variabilidad operativa, con el fin de identificar patrones, inconsistencias y oportunidades de mejora que sirvan de base para las siguientes fases del proyecto CRISP-DM.

Conteo

In [4]:
pd.read_sql("SELECT COUNT(*) AS total_registros FROM transtats_historic", engine)


,total_registros
0,67249397


Campos y tipos de datos

In [5]:
df_columnas = pd.read_sql("""
    SELECT
        column_name,
        data_type
    FROM information_schema.columns
    WHERE table_name = 'transtats_historic'
    ORDER BY ordinal_position;
""", engine)

df_columnas


,column_name,data_type
0,FL_DATE,text
1,OP_UNIQUE_CARRIER,text
2,OP_CARRIER_AIRLINE_ID,text
3,TAIL_NUM,text
4,OP_CARRIER_FL_NUM,text
5,ORIGIN_AIRPORT_ID,text
6,ORIGIN_CITY_MARKET_ID,text
7,ORIGIN,text
8,ORIGIN_STATE_FIPS,text
9,ORIGIN_WAC,text


In [6]:
df_sample = pd.read_sql("""
SELECT *
FROM transtats_historic
TABLESAMPLE SYSTEM (5);
""", engine)

df_sample



df_sample = pd.read_sql("""
SELECT
    FL_DATE,
    OP_UNIQUE_CARRIER,
    ORIGIN,
    DEST,
    AIR_TIME,
    ACTUAL_ELAPSED_TIME,
    CRS_ELAPSED_TIME,
    DISTANCE,
    CARRIER_DELAY,
    WEATHER_DELAY,
    NAS_DELAY,
    SECURITY_DELAY,
    LATE_AIRCRAFT_DELAY
FROM transtats_historic
TABLESAMPLE SYSTEM (2);
""", engine)



,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_STATE_FIPS,ORIGIN_WAC,...,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,DIV_ACTUAL_ELAPSED_TIME,DIV_DISTANCE
0,1/1/2010 12:00:00 AM,9E,20363,85339E,3875,11996,31871,GSP,45,37,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,1/1/2010 12:00:00 AM,9E,20363,85339E,3905,11433,31295,DTW,26,43,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,1/1/2010 12:00:00 AM,9E,20363,85339E,4095,11203,30424,CWA,55,45,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,1/1/2010 12:00:00 AM,9E,20363,85419E,3870,14078,31481,PFN,12,33,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,1/1/2010 12:00:00 AM,9E,20363,85419E,4121,13244,33244,MEM,47,54,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3346802,12/31/2020 12:00:00 AM,YV,20378,N915FJ,5840,14107,30466,PHX,4,81,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3346803,12/31/2020 12:00:00 AM,YV,20378,N915FJ,5840,15023,34433,STS,6,91,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3346804,12/31/2020 12:00:00 AM,YV,20378,N915FJ,5917,14107,30466,PHX,4,81,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3346805,12/31/2020 12:00:00 AM,YV,20378,N915FJ,5998,13198,33198,MCI,29,64,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


Definiendo nulidad

In [7]:
columnas = [
    "FL_DATE",
    "OP_UNIQUE_CARRIER",
    "OP_CARRIER_AIRLINE_ID",
    "TAIL_NUM",
    "OP_CARRIER_FL_NUM",
    "ORIGIN_AIRPORT_ID",
    "ORIGIN_CITY_MARKET_ID",
    "ORIGIN",
    "ORIGIN_STATE_FIPS",
    "ORIGIN_WAC",
    "DEST_AIRPORT_ID",
    "DEST_CITY_MARKET_ID",
    "DEST",
    "DEST_STATE_FIPS",
    "DEST_WAC",
    "CRS_DEP_TIME",
    "DEP_TIME",
    "WHEELS_OFF",
    "WHEELS_ON",
    "CRS_ARR_TIME",
    "ARR_TIME",
    "CANCELLATION_CODE",
    "CRS_ELAPSED_TIME",
    "ACTUAL_ELAPSED_TIME",
    "AIR_TIME",
    "FLIGHTS",
    "DISTANCE",
    "CARRIER_DELAY",
    "WEATHER_DELAY",
    "NAS_DELAY",
    "SECURITY_DELAY",
    "LATE_AIRCRAFT_DELAY",
    "FIRST_DEP_TIME",
    "TOTAL_ADD_GTIME",
    "LONGEST_ADD_GTIME",
    "DIV_ACTUAL_ELAPSED_TIME",
    "DIV_DISTANCE"
]


In [8]:
select_parts = ["SELECT COUNT(*) AS total_registros"]

for col in columnas:
    col_sql = f'"{col}"'
    select_parts.append(
        f", SUM(CASE WHEN {col_sql} IS NULL OR {col_sql} = '' THEN 1 ELSE 0 END) AS {col}_nulos"
    )

sql_full = "\n".join(select_parts) + "\nFROM transtats_historic;"
sql_full


'SELECT COUNT(*) AS total_registros\n, SUM(CASE WHEN "FL_DATE" IS NULL OR "FL_DATE" = \'\' THEN 1 ELSE 0 END) AS FL_DATE_nulos\n, SUM(CASE WHEN "OP_UNIQUE_CARRIER" IS NULL OR "OP_UNIQUE_CARRIER" = \'\' THEN 1 ELSE 0 END) AS OP_UNIQUE_CARRIER_nulos\n, SUM(CASE WHEN "OP_CARRIER_AIRLINE_ID" IS NULL OR "OP_CARRIER_AIRLINE_ID" = \'\' THEN 1 ELSE 0 END) AS OP_CARRIER_AIRLINE_ID_nulos\n, SUM(CASE WHEN "TAIL_NUM" IS NULL OR "TAIL_NUM" = \'\' THEN 1 ELSE 0 END) AS TAIL_NUM_nulos\n, SUM(CASE WHEN "OP_CARRIER_FL_NUM" IS NULL OR "OP_CARRIER_FL_NUM" = \'\' THEN 1 ELSE 0 END) AS OP_CARRIER_FL_NUM_nulos\n, SUM(CASE WHEN "ORIGIN_AIRPORT_ID" IS NULL OR "ORIGIN_AIRPORT_ID" = \'\' THEN 1 ELSE 0 END) AS ORIGIN_AIRPORT_ID_nulos\n, SUM(CASE WHEN "ORIGIN_CITY_MARKET_ID" IS NULL OR "ORIGIN_CITY_MARKET_ID" = \'\' THEN 1 ELSE 0 END) AS ORIGIN_CITY_MARKET_ID_nulos\n, SUM(CASE WHEN "ORIGIN" IS NULL OR "ORIGIN" = \'\' THEN 1 ELSE 0 END) AS ORIGIN_nulos\n, SUM(CASE WHEN "ORIGIN_STATE_FIPS" IS NULL OR "ORIGIN_STATE_

In [9]:
df_raw = pd.read_sql(sql_full, engine)
df_raw


,total_registros,fl_date_nulos,op_unique_carrier_nulos,op_carrier_airline_id_nulos,tail_num_nulos,op_carrier_fl_num_nulos,origin_airport_id_nulos,origin_city_market_id_nulos,origin_nulos,origin_state_fips_nulos,...,carrier_delay_nulos,weather_delay_nulos,nas_delay_nulos,security_delay_nulos,late_aircraft_delay_nulos,first_dep_time_nulos,total_add_gtime_nulos,longest_add_gtime_nulos,div_actual_elapsed_time_nulos,div_distance_nulos
0,67249397,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
pd.set_option('display.max_columns', None)
df_raw


,total_registros,fl_date_nulos,op_unique_carrier_nulos,op_carrier_airline_id_nulos,tail_num_nulos,op_carrier_fl_num_nulos,origin_airport_id_nulos,origin_city_market_id_nulos,origin_nulos,origin_state_fips_nulos,origin_wac_nulos,dest_airport_id_nulos,dest_city_market_id_nulos,dest_nulos,dest_state_fips_nulos,dest_wac_nulos,crs_dep_time_nulos,dep_time_nulos,wheels_off_nulos,wheels_on_nulos,crs_arr_time_nulos,arr_time_nulos,cancellation_code_nulos,crs_elapsed_time_nulos,actual_elapsed_time_nulos,air_time_nulos,flights_nulos,distance_nulos,carrier_delay_nulos,weather_delay_nulos,nas_delay_nulos,security_delay_nulos,late_aircraft_delay_nulos,first_dep_time_nulos,total_add_gtime_nulos,longest_add_gtime_nulos,div_actual_elapsed_time_nulos,div_distance_nulos
0,67249397,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


El data set no contiene columnas con nulidad, por lo tanto no se descartan variables

In [11]:
columnas = [
    "FL_DATE",
    "OP_UNIQUE_CARRIER",
    "OP_CARRIER_AIRLINE_ID",
    "TAIL_NUM",
    "OP_CARRIER_FL_NUM",
    "ORIGIN_AIRPORT_ID",
    "ORIGIN_CITY_MARKET_ID",
    "ORIGIN",
    "ORIGIN_STATE_FIPS",
    "ORIGIN_WAC",
    "DEST_AIRPORT_ID",
    "DEST_CITY_MARKET_ID",
    "DEST",
    "DEST_STATE_FIPS",
    "DEST_WAC",
    "CRS_DEP_TIME",
    "DEP_TIME",
    "WHEELS_OFF",
    "WHEELS_ON",
    "CRS_ARR_TIME",
    "ARR_TIME",
    "CANCELLATION_CODE",
    "CRS_ELAPSED_TIME",
    "ACTUAL_ELAPSED_TIME",
    "AIR_TIME",
    "FLIGHTS",
    "DISTANCE",
    "CARRIER_DELAY",
    "WEATHER_DELAY",
    "NAS_DELAY",
    "SECURITY_DELAY",
    "LATE_AIRCRAFT_DELAY",
    "FIRST_DEP_TIME",
    "TOTAL_ADD_GTIME",
    "LONGEST_ADD_GTIME",
    "DIV_ACTUAL_ELAPSED_TIME",
    "DIV_DISTANCE"
]

total = df_raw.loc[0, "total_registros"]
filas = []

for col in columnas:
    col_alias = f"{col.lower()}_nulos"   # fl_date_nulos, origin_nulos, etc.
    nulos = int(df_raw.loc[0, col_alias])
    pct = round(nulos * 100.0 / total, 4)
    filas.append({
        "columna": col,
        "nulos": nulos,
        "porcentaje_nulos": pct
    })

df_nulos = (
    pd.DataFrame(filas)
      .sort_values("porcentaje_nulos", ascending=False)
      .reset_index(drop=True)
)

df_nulos


,columna,nulos,porcentaje_nulos
0,FL_DATE,0,0.0
1,OP_UNIQUE_CARRIER,0,0.0
2,OP_CARRIER_AIRLINE_ID,0,0.0
3,TAIL_NUM,0,0.0
4,OP_CARRIER_FL_NUM,0,0.0
5,ORIGIN_AIRPORT_ID,0,0.0
6,ORIGIN_CITY_MARKET_ID,0,0.0
7,ORIGIN,0,0.0
8,ORIGIN_STATE_FIPS,0,0.0
9,ORIGIN_WAC,0,0.0


In [12]:
df_sample.columns.tolist()


['FL_DATE',
 'OP_UNIQUE_CARRIER',
 'OP_CARRIER_AIRLINE_ID',
 'TAIL_NUM',
 'OP_CARRIER_FL_NUM',
 'ORIGIN_AIRPORT_ID',
 'ORIGIN_CITY_MARKET_ID',
 'ORIGIN',
 'ORIGIN_STATE_FIPS',
 'ORIGIN_WAC',
 'DEST_AIRPORT_ID',
 'DEST_CITY_MARKET_ID',
 'DEST',
 'DEST_STATE_FIPS',
 'DEST_WAC',
 'CRS_DEP_TIME',
 'DEP_TIME',
 'WHEELS_OFF',
 'WHEELS_ON',
 'CRS_ARR_TIME',
 'ARR_TIME',
 'CANCELLATION_CODE',
 'CRS_ELAPSED_TIME',
 'ACTUAL_ELAPSED_TIME',
 'AIR_TIME',
 'FLIGHTS',
 'DISTANCE',
 'CARRIER_DELAY',
 'WEATHER_DELAY',
 'NAS_DELAY',
 'SECURITY_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'FIRST_DEP_TIME',
 'TOTAL_ADD_GTIME',
 'LONGEST_ADD_GTIME',
 'DIV_ACTUAL_ELAPSED_TIME',
 'DIV_DISTANCE']

¿Qué aeropuertos tienen mayor incidencia de retrasos?

se trabaja sobre una muestra dle 5% del total del data set

In [25]:
df_sample["ARR_TIME"] = pd.to_numeric(df_sample["ARR_TIME"], errors="coerce")
df_sample["CRS_ARR_TIME"] = pd.to_numeric(df_sample["CRS_ARR_TIME"], errors="coerce")

df_sample["delay"] = df_sample["ARR_TIME"] - df_sample["CRS_ARR_TIME"]
df_sample["is_delayed"] = df_sample["delay"] > 0


Proporcion de vuelos retrasados vs los que no se retrasan

In [26]:
df_sample["is_delayed"].value_counts(normalize=True) * 100


,proportion
is_delayed,
False,65.665991
True,34.334009


Top de Aeropuertos con mayor % de vuelos retrasados

In [27]:
df_sample.groupby("ORIGIN")["is_delayed"] \
  .mean() \
  .sort_values(ascending=False) \
  .head(10) * 100


,is_delayed
ORIGIN,
JST,100.000000
IFP,65.384615
CDB,62.500000
FOE,60.714286
PGV,51.923077
STC,50.000000
CEC,48.518519
PPG,47.916667
ADK,47.297297


Top de Aerolineas con menor % de retrasos

In [29]:
df_sample.groupby("OP_UNIQUE_CARRIER")["is_delayed"] \
  .mean() \
  .sort_values() \
  .head(10) * 100


,is_delayed
OP_UNIQUE_CARRIER,
YX,27.099879
9E,27.605460
DL,28.992950
G4,31.953536
AS,31.998418
UA,32.484395
NK,32.614408
YV,33.474941
HA,33.627544


Estacionalidad por mes

In [30]:
df_sample["month"] = pd.to_datetime(df_sample["FL_DATE"]).dt.month

df_sample.groupby("month")["is_delayed"].mean() * 100


/tmp/ipython-input-334199648.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_sample["month"] = pd.to_datetime(df_sample["FL_DATE"]).dt.month


,is_delayed
month,
1,34.525538
2,34.407362
3,33.337159
4,32.698086
5,34.765463
6,38.710243
7,37.236198
8,35.015460
9,29.905251


ANALISIS CORRELACIONAL

In [31]:
df_sample[["delay", "DISTANCE", "AIR_TIME"]].corr()


,delay,DISTANCE,AIR_TIME
delay,1.000000,-0.023267,-0.018090
DISTANCE,-0.023267,1.000000,0.984655
AIR_TIME,-0.018090,0.984655,1.000000
